https://www.kaggle.com/areeves87/rscience-popular-comment-removal/downloads/rscience-popular-comment-removal.zip/3

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.text import *

In [ ]:
df = pd.read_csv('reddit_200k_train.csv', encoding="ISO-8859-1")
df.head()

In [ ]:
df.body[1]

In [ ]:
data_lm.save()

In [ ]:
data = load_data(".")

In [ ]:
valid_pct = 0.2
cut = int(valid_pct * len(df)) + 1
data_lm = (TextLMDataBunch.from_df(path=".",
                                 train_df = df[cut:],
                                 valid_df = df[:cut],
                                 text_cols = "body",
                                 label_cols = "REMOVED")
       )

In [ ]:
data_lm.save()

In [ ]:
data_lm = load_data(".")

In [ ]:
data_lm.show_batch()

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.load('fit_head');

In [ ]:
learn.fit_one_cycle(10, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned')

In [ ]:
TEXT = "Newton's first law has nothing to do with"
N_WORDS = 40
N_SENTENCES = 2

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))

In [ ]:
learn.save_encoder('fine_tuned_enc')

In [ ]:
!jupyter nbconvert rscience-comment-removal.ipynb --to html --output nbs/2019-07-02_lm_only